<a href="https://colab.research.google.com/github/pranathiir/pistop-analysis/blob/main/pitstopAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# setup



In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import os

filenames = []
filepaths = []
for dirname, _, fnames in os.walk('/content/drive/MyDrive/pitstop data'):
  for filename in fnames:
    filenames.append(filename.split('.')[0])
    filepaths.append(os.path.join(dirname, filename))
    print(os.path.join(dirname, filename))

/content/drive/MyDrive/pitstop data/circuits.csv
/content/drive/MyDrive/pitstop data/constructor_results.csv
/content/drive/MyDrive/pitstop data/constructor_standings.csv
/content/drive/MyDrive/pitstop data/drivers.csv
/content/drive/MyDrive/pitstop data/driver_standings.csv
/content/drive/MyDrive/pitstop data/races.csv
/content/drive/MyDrive/pitstop data/pit_stops.csv
/content/drive/MyDrive/pitstop data/qualifying.csv
/content/drive/MyDrive/pitstop data/seasons.csv
/content/drive/MyDrive/pitstop data/sprint_results.csv
/content/drive/MyDrive/pitstop data/status.csv
/content/drive/MyDrive/pitstop data/lap_times.csv
/content/drive/MyDrive/pitstop data/results.csv
/content/drive/MyDrive/pitstop data/constructors.csv


#reading data

In [ ]:
fpath = '/content/drive/MyDrive/pitstop data/'
circuits = pd.read_csv(f'{fpath}circuits.csv', index_col=0, na_values=r'\N')
constructorResults = pd.read_csv(f'{fpath}constructor_results.csv', index_col=0, na_values=r'\N')
constructors = pd.read_csv(f'{fpath}constructors.csv', index_col=0, na_values=r'\N')
constructorStandings = pd.read_csv(f'{fpath}constructor_standings.csv', index_col=0, na_values=r'\N')
drivers = pd.read_csv(f'{fpath}drivers.csv', index_col=0, na_values=r'\N')
driverStandings = pd.read_csv(f'{fpath}driver_standings.csv', index_col=0, na_values=r'\N')
lapTimes = pd.read_csv(f'{fpath}lap_times.csv')
pitStops = pd.read_csv(f'{fpath}pit_stops.csv')
qualifying = pd.read_csv(f'{fpath}qualifying.csv', index_col=0, na_values=r'\N')
races = pd.read_csv(f'{fpath}races.csv', na_values=r'\N')
results = pd.read_csv(f'{fpath}results.csv', index_col=0, na_values=r'\N')
seasons = pd.read_csv(f'{fpath}seasons.csv', index_col=0, na_values=r'\N')
status = pd.read_csv(f'{fpath}status.csv', index_col=0, na_values=r'\N')

circuits = circuits.rename(columns={'name': 'circuitName', 'location': 'circuitLocation', 'country': 'circuitCountry', 'url': 'circuitURL'})
drivers = drivers.rename(columns={'nationality': 'driverNationality', 'url': 'driverURL'})
drivers['driverName'] = drivers['forename'] + ' ' + drivers['surname']
constructors = constructors.rename(columns={'name': 'constructorName', 'nationality': 'constructorNnationality', 'url': 'constructorURL'})
races.index = races.index.set_names(['raceId','year','round','circuitId','raceName','date','time','raceUrl','a','b'])
races = races[[]].reset_index()[['raceId','year','round','circuitId','raceName','date','time','raceUrl']]
races.set_index('raceId',inplace=True)
races['date'] = races['date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))
pitStops = pitStops.rename(columns={'time':'pitTime'})
pitStops['seconds'] = pitStops['milliseconds'].apply(lambda x: x/1000)
results['seconds'] = results['milliseconds'].apply(lambda x: x/1000)


In [ ]:
#constructors color mapping
constructor_color_map = {
    'Toro Rosso':'#0000FF',
    'Mercedes':'#6CD3BF',
    'Red Bull':'#1E5BC6',
    'Ferrari':'#ED1C24',
    'Williams':'#37BEDD',
    'Force India':'#FF80C7',
    'Virgin':'#c82e37',
    'Renault':'#FFD800',
    'McLaren':'#F58020',
    'Sauber':'#006EFF',
    'Lotus':'#FFB800',
    'HRT':'#b2945e',
    'Caterham':'#0b361f',
    'Lotus F1':'#FFB800',
    'Marussia':'#6E0000',
    'Manor Marussia':'#6E0000',
    'Haas F1 Team':'#B6BABD',
    'Racing Point':'#F596C8',
    'Aston Martin':'#2D826D',
    'Alfa Romeo':'#B12039',
    'AlphaTauri':'#4E7C9B',
    'Alpine F1 Team':'#2293D1'
}

In [ ]:
#pitStops
pitStops.describe().T

newResults = pd.merge(results, races,left_on='raceId', right_index=True, how='left')
newResults = pd.merge(newResults, circuits, left_on='circuitId', right_index=True, how='left')
newResults = pd.merge(newResults, constructors, left_on='constructorId', right_index=True, how='left')
newResults = pd.merge(newResults, drivers, left_on='driverId', right_index=True, how='left')
newResults

,raceId,driverId,constructorId,number_x,grid,position,positionText,positionOrder,points,laps,...,constructorURL,driverRef,number_y,code,forename,surname,dob,driverNationality,driverURL,driverName
resultId,,,,,,,,,,,,,,,,,,,,,
1,18,1,1,22.0,1,1.0,1,1,10.0,58,...,http://en.wikipedia.org/wiki/McLaren,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,Lewis Hamilton
2,18,2,2,3.0,5,2.0,2,2,8.0,58,...,http://en.wikipedia.org/wiki/BMW_Sauber,heidfeld,NaN,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,Nick Heidfeld
3,18,3,3,7.0,7,3.0,3,3,6.0,58,...,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,rosberg,6.0,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg,Nico Rosberg
4,18,4,4,5.0,11,4.0,4,4,5.0,58,...,http://en.wikipedia.org/wiki/Renault_in_Formul...,alonso,14.0,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,Fernando Alonso
5,18,5,1,23.0,3,5.0,5,5,4.0,58,...,http://en.wikipedia.org/wiki/McLaren,kovalainen,NaN,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen,Heikki Kovalainen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25421,1074,849,3,6.0,20,16.0,16,16,0.0,57,...,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,latifi,6.0,LAT,Nicholas,Latifi,1995-06-29,Canadian,http://en.wikipedia.org/wiki/Nicholas_Latifi,Nicholas Latifi
25422,1074,807,117,27.0,17,17.0,17,17,0.0,57,...,http://en.wikipedia.org/wiki/Aston_Martin_in_F...,hulkenberg,27.0,HUL,Nico,Hülkenberg,1987-08-19,German,http://en.wikipedia.org/wiki/Nico_H%C3%BClkenberg,Nico Hülkenberg
25423,1074,815,9,11.0,4,18.0,18,18,0.0,56,...,http://en.wikipedia.org/wiki/Red_Bull_Racing,perez,11.0,PER,Sergio,Pérez,1990-01-26,Mexican,http://en.wikipedia.org/wiki/Sergio_P%C3%A9rez,Sergio Pérez


In [ ]:
newPitStops = pd.merge(pitStops, races, left_on='raceId', right_index=True, how='left')
newPitStops = pd.merge(newPitStops, circuits, left_on='circuitId', right_index=True, how='left')
newPitStops = pd.merge(newPitStops, newResults[['raceId', 'driverId', 'driverName', 'constructorId', 'constructorName']], left_on=['raceId', 'driverId'], right_on=['raceId', 'driverId'])
newPitStops

,raceId,driverId,stop,lap,pitTime,duration,milliseconds,seconds,year,round,...,circuitName,circuitLocation,circuitCountry,lat,lng,alt,circuitURL,driverName,constructorId,constructorName
0,841,153,1,1,17:05:23,26.898,26898,26.898,2011,1,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Jaime Alguersuari,5,Toro Rosso
1,841,153,2,17,17:31:06,24.463,24463,24.463,2011,1,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Jaime Alguersuari,5,Toro Rosso
2,841,153,3,35,17:59:45,26.348,26348,26.348,2011,1,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Jaime Alguersuari,5,Toro Rosso
3,841,30,1,1,17:05:52,25.021,25021,25.021,2011,1,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Michael Schumacher,131,Mercedes
4,841,30,2,17,17:32:08,23.988,23988,23.988,2011,1,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Michael Schumacher,131,Mercedes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8882,1074,807,2,37,19:06:47,26.712,26712,26.712,2022,1,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7.0,http://en.wikipedia.org/wiki/Bahrain_Internati...,Nico Hülkenberg,117,Aston Martin
8883,1074,807,3,44,19:18:58,25.748,25748,25.748,2022,1,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7.0,http://en.wikipedia.org/wiki/Bahrain_Internati...,Nico Hülkenberg,117,Aston Martin
8884,1074,846,1,24,18:44:41,24.568,24568,24.568,2022,1,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7.0,http://en.wikipedia.org/wiki/Bahrain_Internati...,Lando Norris,1,McLaren
8885,1074,846,2,41,19:13:23,27.367,27367,27.367,2022,1,...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7.0,http://en.wikipedia.org/wiki/Bahrain_Internati...,Lando Norris,1,McLaren


In [ ]:
raceResults = pd.merge(newResults, newPitStops.groupby(by=['raceId', 'raceName', 'constructorName', 'driverId', 'driverName']).sum(), left_on=['raceId', 'driverId'], right_on=['raceId', 'driverId'], how='left')
raceResults['pitPercentage'] = raceResults['milliseconds_y']/raceResults['milliseconds_x']*100
raceResults

<ipython-input-14-095ee7163843>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  raceResults = pd.merge(newResults, newPitStops.groupby(by=['raceId', 'raceName', 'constructorName', 'driverId', 'driverName']).sum(), left_on=['raceId', 'driverId'], right_on=['raceId', 'driverId'], how='left')


,raceId,driverId,constructorId_x,number_x,grid,position,positionText,positionOrder,points,laps,...,milliseconds_y,seconds_y,year_y,round_y,circuitId_y,lat_y,lng_y,alt_y,constructorId_y,pitPercentage
0,18,1,1,22.0,1,1.0,1,1,10.0,58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18,2,2,3.0,5,2.0,2,2,8.0,58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18,3,3,7.0,7,3.0,3,3,6.0,58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,4,4,5.0,11,4.0,4,4,5.0,58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18,5,1,23.0,3,5.0,5,5,4.0,58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25415,1074,849,3,6.0,20,16.0,16,16,0.0,57,...,73893.0,73.893,6066.0,3.0,9.0,78.0975,151.5318,21.0,9.0,1.249168
25416,1074,807,117,27.0,17,17.0,17,17,0.0,57,...,79825.0,79.825,6066.0,3.0,9.0,78.0975,151.5318,21.0,351.0,1.348985
25417,1074,815,9,11.0,4,18.0,18,18,0.0,56,...,74166.0,74.166,6066.0,3.0,9.0,78.0975,151.5318,21.0,27.0,NaN
25418,1074,830,9,1.0,2,19.0,19,19,0.0,54,...,74236.0,74.236,6066.0,3.0,9.0,78.0975,151.5318,21.0,27.0,NaN


In [ ]:
fig = px.line(newPitStops[newPitStops['seconds']<50].groupby(by=['year', 'constructorName']).mean().reset_index(),
                 x='year',
                 y='seconds',
                 color='constructorName',
                 color_discrete_map=constructor_color_map,
                )
fig.update_layout(
    title_text='Average Pit Stop Durations by Constructor',
)
fig.show()

In [ ]:
#scatter plot
fig = px.scatter(newPitStops[newPitStops['seconds']<50],
                 x='date',
                 y='seconds',
                 color='constructorName',
                 color_discrete_map=constructor_color_map,
                )
fig.update_layout(
    title_text='Pit Stop Durations over Time by Constructor',
)
fig.show()
